# Segmenting and Clustering Neighborhoods in Toronto

Required to explore and cluster the neighborhoods in Toronto.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, features='html')
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [3]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string.rstrip());      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1].rstrip())
                else:
                    neighborhoods.append(cell.string.rstrip()); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [4]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [6]:
# initialize your variable to None
lat_lng_coords = None

for k in range(0, len(postalCodes)-1):
    code = postalCodes[k]
    borough = boroughs[k]
    neighborhood_name = neighborhoods[k]
    
    print('{}/{} - {}'.format(k,len(postalCodes)-1,code))
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

0/209 - M3A
1/209 - M4A
2/209 - M5A
3/209 - M6A
4/209 - M6A
5/209 - M7A
6/209 - M9A
7/209 - M1B
8/209 - M1B
9/209 - M3B
10/209 - M4B
11/209 - M4B
12/209 - M5B
13/209 - M5B
14/209 - M6B
15/209 - M9B
16/209 - M9B
17/209 - M9B
18/209 - M9B
19/209 - M9B
20/209 - M1C
21/209 - M1C
22/209 - M1C
23/209 - M3C
24/209 - M3C
25/209 - M4C
26/209 - M5C
27/209 - M6C
28/209 - M9C
29/209 - M9C
30/209 - M9C
31/209 - M9C
32/209 - M1E
33/209 - M1E
34/209 - M1E
35/209 - M4E
36/209 - M5E
37/209 - M6E
38/209 - M1G
39/209 - M4G
40/209 - M5G
41/209 - M6G
42/209 - M1H
43/209 - M2H
44/209 - M3H
45/209 - M3H
46/209 - M3H
47/209 - M4H
48/209 - M5H
49/209 - M5H
50/209 - M5H
51/209 - M6H
52/209 - M6H
53/209 - M1J
54/209 - M2J
55/209 - M2J
56/209 - M2J
57/209 - M3J
58/209 - M3J
59/209 - M4J
60/209 - M5J
61/209 - M5J
62/209 - M5J
63/209 - M6J
64/209 - M6J
65/209 - M1K
66/209 - M1K
67/209 - M1K
68/209 - M2K
69/209 - M3K
70/209 - M3K
71/209 - M4K
72/209 - M4K
73/209 - M5K
74/209 - M5K
75/209 - M6K
76/209 - M6K
77/209 - 

# Choosing the following neighborhood:

In [48]:
neighbors[neighbors['Neighborhood']=='Church and Wellesley']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
299,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302


In [49]:
VERSION = '20180605' # Foursquare API version

CLIENT_ID = '1HWJZFVUJPVG5Z3JLOSPFNW42I4G0B4LKSXQZNWNHS2JSXDQ'
CLIENT_SECRET = 'KLIZ4SNGKISIRARYRB4RFPXTE2BTCASV2WHELYBREUITLY5F'

idx=299
neighborhood_name = neighbors.iloc[idx].Neighborhood # neighborhood name
neighborhood_latitude = neighbors.iloc[299].Latitude # neighborhood latitude value
neighborhood_longitude = neighbors.iloc[299].Longitude # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 1000 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

print(url)

results = requests.get(url).json()

Latitude and longitude values of Church and Wellesley are 43.666585000000055, -79.38130203699995.
https://api.foursquare.com/v2/venues/explore?&client_id=1HWJZFVUJPVG5Z3JLOSPFNW42I4G0B4LKSXQZNWNHS2JSXDQ&client_secret=KLIZ4SNGKISIRARYRB4RFPXTE2BTCASV2WHELYBREUITLY5F&v=20180605&ll=43.666585000000055,-79.38130203699995&radius=1000&limit=100


In [50]:
# From the Foursquare lab in the previous module, we know that all the information is in the items key. 
# Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [51]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Storm Crow Manor,Theme Restaurant,43.666840,-79.381593
1,Smith,Breakfast Spot,43.666927,-79.381421
2,The Keg Steakhouse + Bar,Steakhouse,43.666756,-79.378302
3,Fabarnak,Restaurant,43.666377,-79.380964
4,Ho's Team Barber Shop,Salon / Barbershop,43.665630,-79.381359


In [52]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [53]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Queen's Park
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toronto 

In [55]:
# check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(21000, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75242,-79.329242,Storm Crow Manor,43.666840,-79.381593,Theme Restaurant
1,Parkwoods,43.75242,-79.329242,Smith,43.666927,-79.381421,Breakfast Spot
2,Parkwoods,43.75242,-79.329242,The Keg Steakhouse + Bar,43.666756,-79.378302,Steakhouse
3,Parkwoods,43.75242,-79.329242,Fabarnak,43.666377,-79.380964,Restaurant
4,Parkwoods,43.75242,-79.329242,Ho's Team Barber Shop,43.665630,-79.381359,Salon / Barbershop


In [56]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,100,100,100,100,100,100
Agincourt North,100,100,100,100,100,100
Albion Gardens,100,100,100,100,100,100
Alderwood,100,100,100,100,100,100
...,...,...,...,...,...,...
Woodbine Heights,100,100,100,100,100,100
York Mills,100,100,100,100,100,100
York Mills West,100,100,100,100,100,100


In [57]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 67 uniques categories.


In [58]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Beer Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,...,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Theme Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Shop,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# examine the new dataframe size.

toronto_onehot.shape

(21000, 68)

In [60]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Beer Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,...,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Theme Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Shop,Yoga Studio
0,Adelaide,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,...,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01
1,Agincourt,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,...,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01
2,Agincourt North,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,...,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01
3,Albion Gardens,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,...,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01
4,Alderwood,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,...,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Woodbine Heights,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,...,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01
203,York Mills,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,...,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01
204,York Mills West,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,...,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01
205,York University,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,...,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01


In [61]:
# confirm the new size

toronto_grouped.shape

(207, 68)

In [62]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Agincourt----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Agincourt North----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Albion Gardens----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Alderwood----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Bathurst Manor--

                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----East Birchmount Park----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----East Toronto----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Emery----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Eringate----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Exhibition Place----
              

                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Milliken----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Mimico NE----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Mimico NW----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Mimico South----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Montgomery Road----
                 venue

                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----Silverthorn----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----South Hill----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----South Niagara----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----South Steeles----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.05
2           Restaurant  0.03
3                  Gym  0.03
4                 Park  0.03


----South of Bloor----
              

In [63]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Adelaide,Coffee Shop,Japanese Restaurant,Gym
1,Agincourt,Coffee Shop,Japanese Restaurant,Gym
2,Agincourt North,Coffee Shop,Japanese Restaurant,Gym
3,Albion Gardens,Coffee Shop,Japanese Restaurant,Gym
4,Alderwood,Coffee Shop,Japanese Restaurant,Gym


In [72]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

<ipython-input-72-8a42a5234458>:9: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [68]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,0,Coffee Shop,Japanese Restaurant,Gym
1,M4A,North York,Victoria Village,43.730600,-79.313265,0,Coffee Shop,Japanese Restaurant,Gym
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,0,Coffee Shop,Japanese Restaurant,Gym
3,M6A,North York,Lawrence Heights,43.723270,-79.451286,0,Coffee Shop,Japanese Restaurant,Gym
4,M6A,North York,Lawrence Manor,43.723270,-79.451286,0,Coffee Shop,Japanese Restaurant,Gym


In [69]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [70]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,North York,0,Coffee Shop,Japanese Restaurant,Gym
1,North York,0,Coffee Shop,Japanese Restaurant,Gym
2,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gym
3,North York,0,Coffee Shop,Japanese Restaurant,Gym
4,North York,0,Coffee Shop,Japanese Restaurant,Gym
...,...,...,...,...,...
308,Etobicoke,0,Coffee Shop,Japanese Restaurant,Gym
309,Etobicoke,0,Coffee Shop,Japanese Restaurant,Gym
310,Etobicoke,0,Coffee Shop,Japanese Restaurant,Gym
311,Etobicoke,0,Coffee Shop,Japanese Restaurant,Gym
